Data Labeling
===============================
## Description
In this notebook, the labels for the data are created so that supervised learning algorithms can be trained and tested. This is done by applying expert-domain rules based on the damping factor $\zeta$. The first part is to import the required libraries and the raw eigenvalues.

In [6]:
%run -i import_libraries.py

### Importing Raw Eigenvalues

In [7]:
# Loading simulation results
eigs19branches = np.load("99_Data/19Branches_all_eigs.npy")
print(f"We have {eigs19branches.shape[0]} number of scenarios, each with {eigs19branches.shape[1]} eigenvalues")

current_wd = os.getcwd()

if not os.path.exists(os.path.join(current_wd, "99_Data/df_eigenvalues.pkl")):
    
    print("Creating dataframe...")
    # Creating column labels
    col_labels = []
    for i in range(1, eigs19branches.shape[0] + 1):
        col_labels.append("Scenario {}".format(i))

    # Organizing the data in Pandas
    df_eigenvalues = pd.DataFrame([], columns = col_labels)

    for sc in range(1, eigs19branches.shape[0] + 1):
        df_eigenvalues["Scenario {}".format(sc)] = eigs19branches[(sc - 1)][:]

    # The DataFrame has not been created in memory - Saving it as a .pkl file
    with open("99_Data/df_eigenvalues.pkl", 'wb') as f:
        pickle.dump(df_eigenvalues, f, pickle.HIGHEST_PROTOCOL)
    print("Dataframe created!")
    
if os.path.exists(os.path.join(current_wd, "99_Data/df_eigenvalues.pkl")):
    # DataFrame exists as a .pkl file - Loading it
    with open("99_Data/df_eigenvalues.pkl", 'rb') as f:
        df_eigenvalues = pickle.load(f)
    print("Dataframe loaded!")

We have 19815 number of scenarios, each with 49 eigenvalues
Dataframe loaded!


### Normalizing Eigenvalues (Data Cleaning)

Given the significative irregularity of the eigenvalues, it is necessary to perform some data cleaning to bring them to a form where the most important information is preserved. The relevant information about the power system operating condition specified by a given eigenvalue is contained in the _angle_ it forms with the positive real-axis. Hence, a magnitude normalization seems to be more convenient for this case. 

Despite this, if the real-part of the eigenvalue is positive, it is _unstable_. In other words, it describes an unstable operating condition of the power system. A standard normalization will map the real eigenvalues close to the imaginary axis (i.e., those having a small negative real-part) to $-1+0j$. This is not desirable since it hiddens the fact that the eigenvalue is close to the region of instability. For this reason, the normalization procedure carried in the cell below is an ad-hoc solution that normalizes only the eigenvalues outside of the unit circle in the complex plane: that is, only the eigenvalues whose magnitude is strictly larger or equal than one are normalized.

In [8]:
current_wd = os.getcwd()

if not os.path.exists(os.path.join(current_wd, "99_Data/df_normalized_eigenvalues.pkl")):

    print("Normalizing eigenvalues...")
    # Creating new Pandas DataFrame for normalized eigenvalues
    df_normalized_eigenvalues = pd.DataFrame([], columns = \
                                             list(df_eigenvalues.columns.values))

    # Lambda expression to normalize only the eigenvalues outside the unit circle
    norm_scalar = lambda eig : (eig / np.sqrt(np.power(eig.real, 2) + \
                                              np.power(eig.imag, 2))
                        if np.sqrt(np.power(eig.real, 2) + np.power(eig.imag, 2)) >= 1
                        else eig)

    # Vectorizing the lambda expresion defined before
    norm_vector = np.vectorize(norm_scalar)

    # Normalizing eigenvalues
    for sc in list(df_eigenvalues.columns.values):
        df_normalized_eigenvalues[sc] = norm_vector(df_eigenvalues[sc][:])

    # Saving normalized eigenvalues
    with open("99_Data/df_normalized_eigenvalues.pkl", 'wb') as f:
        pickle.dump(df_normalized_eigenvalues, f, pickle.HIGHEST_PROTOCOL)
    print("Normalized eigenvalues saved!")
    
if os.path.exists(os.path.join(current_wd, "99_Data/df_normalized_eigenvalues.pkl")):
    # DataFrame exists as a .pkl file - Loading it
    with open("99_Data/df_normalized_eigenvalues.pkl", 'rb') as f:
        df_normalized_eigenvalues = pickle.load(f)
    print("Normalized eigenvalues dataframe loaded!")

Normalized eigenvalues dataframe loaded!


### Creating Ground Truth

The labeling is carried out by identifying whether an eigenvalue is real- and complex-valued on the first place. Then, depending on whether the eigenvalue lies on the right-half or left-half plane (positive or negative real part), it is labeled as real/complex unstable or stable, respectively. All unstable eigenvalues are assigned a negative damping factor arbitrarily ($\zeta = -1.1$). For stable complex-valued eigenvalues, we compute the damping factor using a closed-form expression given by

$$\begin{equation}
    \zeta_i = -\frac{\text{Re}\{\lambda_i\})}{\sqrt{(\text{Re}\{\lambda_i\})^2 + (\text{Im}\{\lambda_i\})^2}}
\end{equation}$$

where $\text{Re}\{\lambda_i\}$ and $\text{Im}\{\lambda_i\}$ are the real and imaginary parts of the eigenvalue $\lambda_i$. Stable real-valued eigenvalues have a $\zeta$ larger than 1. Therefore, whenever an eigenvalue is identified as being stable and real, a damping factor $\zeta = 1.1$ is assigned to it.

Once the damping ratio is computed, we use it to create the labels of each of the categories, namely:
* **Unstable** ($\zeta < 0$): the eigenvalue lies on the right-half plane.
* **Stable but critical ($0 \leq \zeta < 0.05$)**: the eigenvalue is stable. However, it exhibits light damping (since the damping ratio is small). Therefore, there could be significant oscillations in the system which is an undesired behavior and would require a corrective action.
* **Acceptable operation ($0.05 \leq \zeta \leq 0.1$)**: this operation corresponds to the scenario in which an eigenvalue is not as close to the imaginary axis and/or the oscillations caused by it do not compromise the stability of the system.
* **Good operation ($0.1 \leq \zeta < 1$)**: the response caused by the eigenvalue does not compromise the current operating condition of the system.
* **Satisfactory operation ($\zeta \geq 1$)**: the damping of the eigenvalue is large enough so that no oscillation will be caused on the system. 

The labeling results are stored in an array called `tag_label`. The function below employs the aforementioned damping ratio-based criterion to label the eigenvalues into the five pre-established categories.

In [13]:
def create_ground_truth(df_eigenvalues, current_wd, df_name):
    
    # Path of the target file for storing the ground truth data
    path_output = os.path.join(current_wd, "99_Data/{}_ground_truth_data.pkl".format(df_name))

    if not os.path.exists(path_output):
        
        t_0 = time.time() # Initializing time measurement of algorithm execution
        
        # Number of scenarios
        n_scenarios = df_eigenvalues.shape[1]
        # Number of eigenvalues per scenario
        n_eigs = df_eigenvalues.shape[0]

        # Name of each scenario
        scenarios = list(df_eigenvalues.columns)

        # Creating label container
        label = np.empty((n_eigs, n_scenarios), dtype = object)
        # Creating damping ratio container
        damping_ratio = np.zeros((n_eigs, n_scenarios))
        # Creating tag (i.e., category) container
        tag_label = np.zeros((n_eigs, n_scenarios))

        for n_sc, sc in enumerate(scenarios):
            for n_ev in range(0, n_eigs):
                # Classification between real and complex conjugate eigenvalues
                if (np.imag(df_eigenvalues[sc][n_ev]) == 0):
                    label[n_ev][n_sc] = "Real,"
                else:
                    if (np.real(df_eigenvalues[sc][n_ev]) == 0):
                        label[n_ev][n_sc] = "Pure imaginary,"
                    else:
                        label[n_ev][n_sc] = "Complex conjugate," 

                # Classification between stable and unstable eigenvalues
                if (np.real(df_eigenvalues[sc][n_ev]) > 0):
                    label[n_ev][n_sc] = label[n_ev][n_sc] + " unstable"
                else:
                    label[n_ev][n_sc] = label[n_ev][n_sc] + " stable"

                # Computation of damping ratio
                if (label[n_ev][n_sc] == "Complex conjugate, unstable" or label[n_ev][n_sc] == "Complex conjugate, stable"):
                    damping_ratio[n_ev][n_sc] = -np.real(df_eigenvalues[sc][n_ev]) / np.sqrt(np.square(np.real(df_eigenvalues[sc][n_ev])) + np.square(np.imag(df_eigenvalues[sc][n_ev])))
                else:
                    if (label[n_ev][n_sc] == "Real, stable"):
                        damping_ratio[n_ev][n_sc] = 1.1
                    else:
                        damping_ratio[n_ev][n_sc] = -1.1

                # Creating labels based on the damping ratio
                if (damping_ratio[n_ev][n_sc] < 0):
                    tag_label[n_ev][n_sc] = 1 # 1 - Unstable operation
                elif (damping_ratio[n_ev][n_sc] < 0.05):
                    tag_label[n_ev][n_sc] = 2 # 2 - Stable but critical operation
                elif (damping_ratio[n_ev][n_sc] >= 0.05 and damping_ratio[n_ev][n_sc] < 0.1):
                    tag_label[n_ev][n_sc] = 3 # 3 - Acceptable operation
                elif (damping_ratio[n_ev][n_sc] >= 0.1 and damping_ratio[n_ev][n_sc] < 1.0):
                    tag_label[n_ev][n_sc] = 4 # Good operation
                elif (np.abs(np.real(df_eigenvalues[sc][n_ev])) < 0.5 and damping_ratio[n_ev][n_sc] >= 1.0):
                    if np.abs(np.real(df_eigenvalues[sc][n_ev])) < 0.01:
                        tag_label[n_ev][n_sc] = 6 # Not relevant
                    else:
                        tag_label[n_ev][n_sc] = 2 # Critical operation; real eigenvalue close to the imaginary axis
                else:
                    tag_label[n_ev][n_sc] = 5 # Satisfactory operation

        # Saving ground truth data of eigenvalues    
        ground_truth_data = {'text_label' : label,
                          'damping_ratio' : damping_ratio,
                          'tag_label' : tag_label}

        with open(path_output, 'wb') as f:
            pickle.dump(ground_truth_data, f, pickle.HIGHEST_PROTOCOL)
        print("Ground truth data saved!\n")

        t_f = time.time() # Finishing time measurement of algorithm execution
        
        print("Time elapsed: {} s".format(t_f-t_0))
        print("{} Eigenvalues (shape): {} (eigs) x {} (scenarios)".format(df_name, df_eigenvalues.shape[0], df_eigenvalues.shape[1]))
        print("{} Text labels (shape): {} (eigs) x {} (scenarios)".format(df_name, label.shape[0], label.shape[1]))
        print("{} Damping ratio (shape): {} (eigs) x {} (scenarios)".format(df_name, damping_ratio.shape[0], damping_ratio.shape[1]))
        print("{} Tag label (shape): {} (eigs) x {} (scenarios)\n".format(df_name, tag_label.shape[0], tag_label.shape[1]))
    else:
        print("Ground truth already exists!\n")

In [12]:
create_ground_truth(df_eigenvalues, current_wd, 'raw')
create_ground_truth(df_normalized_eigenvalues, current_wd, 'normalized')

Ground truth data saved!

Time elapsed: 46.803407430648804 s
raw Eigenvalues (shape): 49 (eigs) x 19815 (scenarios)
raw Text labels (shape): 49 (eigs) x 19815 (scenarios)
raw Damping ratio (shape): 49 (eigs) x 19815 (scenarios)
raw Tag label (shape): 49 (eigs) x 19815 (scenarios)

Ground truth data saved!

Time elapsed: 47.08603644371033 s
normalized Eigenvalues (shape): 49 (eigs) x 19815 (scenarios)
normalized Text labels (shape): 49 (eigs) x 19815 (scenarios)
normalized Damping ratio (shape): 49 (eigs) x 19815 (scenarios)
normalized Tag label (shape): 49 (eigs) x 19815 (scenarios)

